In [11]:
import pandas as pd

# Load the dataset
file_path = 'hotel/Datafiniti_Hotel_Reviews_Jun19.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
data.head(2)

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AWE2FvX5RxPSIh2RscTK,2018-01-18T18:43:12Z,2019-05-20T23:55:47Z,5620 Calle Real,"Hotels,Hotels and motels,Hotel and motel mgmt....",Accommodation & Food Services,Goleta,US,us/ca/goleta/5620callereal/-1127060008,34.44178,...,2018-01-03T00:00:00Z,3,https://www.tripadvisor.com/Hotel_Review-g3243...,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,San Jose,UnitedStates,tatsurok2018,https://www.tripadvisor.com/Hotel_Review-g3243...,https://www.bestwestern.com/en_US/book/hotel-r...
1,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.55722,...,2016-10-09T00:00:00Z,4,https://www.tripadvisor.com/Hotel_Review-g3217...,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,San Francisco,CA,STEPHEN N,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com


In [12]:
data_dev = data[['reviews.text', 'reviews.rating']]

data_dev.head()

,reviews.text,reviews.rating
0,"This hotel was nice and quiet. Did not know, t...",3
1,We stayed in the king suite with the separatio...,4
2,"Parking was horrible, somebody ran into my ren...",3
3,Not cheap but excellent location. Price is som...,5
4,If you get the room that they advertised on th...,2


In [13]:
# Update the function to handle non-string inputs gracefully
def is_gibberish(text):
    if pd.isna(text) or not isinstance(text, str):
        return False  # Treat non-strings and NaNs as not gibberish for this purpose
    return any(ord(char) > 127 for char in text)

# Re-apply the updated function to identify gibberish
data_dev['gibberish'] = data_dev['reviews.text'].apply(is_gibberish)

# Recount gibberish rows
gibberish_count_updated = data_dev['gibberish'].sum()

# Remove rows identified as gibberish again
data_cleaned_updated = data_dev[~data_dev['gibberish']].drop(columns='gibberish')

# Remove duplicates again
data_cleaned_updated = data_cleaned_updated.drop_duplicates()

# Display the updated gibberish count and the shape of the cleaned dataset
gibberish_count_updated, data_cleaned_updated.shape


C:\Users\jrzcn\AppData\Local\Temp\ipykernel_9752\385785719.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_dev['gibberish'] = data_dev['reviews.text'].apply(is_gibberish)


(0, (9774, 2))

In [14]:
data_cleaned_updated.to_csv('hotel/test.csv', index=False)

In [15]:
# Load the three datasets: dev, train, and test
file_paths = {
    'dev': 'hotel/dev.csv',
    'train': 'hotel/train.csv',
    'test': 'hotel/test.csv'  # Assuming .dsv is a typo and should be .csv
}

dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

# Since we're looking for duplicates with at least 5 words, we need to filter comments accordingly
def filter_short_comments(df, min_words=5):
    """Filter out comments with fewer than min_words words."""
    is_long_enough = df['reviews.text'].apply(lambda x: len(str(x).split()) >= min_words)
    return df[is_long_enough]

# Apply the filter to each dataset
filtered_dfs = {name: filter_short_comments(df) for name, df in dataframes.items()}

# Concatenate all filtered comments to find duplicates across datasets
all_comments_combined = pd.concat(filtered_dfs.values(), ignore_index=True)
duplicates = all_comments_combined[all_comments_combined.duplicated(['reviews.text'], keep=False)]

# Show the number of duplicates found (if any) and their details
duplicates_count = duplicates.shape[0]
duplicates_details = duplicates if duplicates_count > 0 else "No duplicates found with the criteria."

duplicates_count, duplicates_details


(1146,
                                             reviews.text  reviews.rating
 33                                   It was a great stay             5.0
 96                       Bad: The view. Good: Everything             5.0
 1067   Walked in to the room and was greeted by an ov...             2.0
 1135   The room was so noisy we could hear every foot...             2.0
 1214   Steve the Bartender was very knowledgable abou...             5.0
 ...                                                  ...             ...
 52239  We stayed 2 nights at this motel. It was a pre...             3.0
 52255  We pulled in for the night on a long trip. Bed...             4.0
 52257  My stay at the hotel was only one night but it...             5.0
 52323  This is one of only a handful of times I have ...             4.0
 52474  I stayed at this Motel 6, 311 Courtland St NE,...             5.0
 
 [1146 rows x 2 columns])

In [18]:
import pandas as pd

# Load the uploaded file
file_path = 'hotel/train.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
data.head(), data.isnull().sum()



AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
data_cleaned = data.dropna()

# Save the cleaned data back to the original file
cleaned_file_path = 'hotel/train.csv'
data_cleaned.to_csv(cleaned_file_path, index=False)

cleaned_file_path

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict


data = pd.read_csv('hotel/dev.csv')

# 加载英文停用词
stop_words = set(stopwords.words('english'))

# 扩展缩写词的映射表
contractions = {
    "'s": "is",
    "'ll": "will",
    "'d": "would"
    # 可以根据需要添加更多
}

# 预处理和标记化评论文本
def preprocess_text(text):
    # 将缩写替换为完整形式
    for contraction, full_form in contractions.items():
        text = text.replace(contraction, full_form)
    # 标记化
    words = word_tokenize(text)
    # 去除停用词并且过滤非字母字符
    words_filtered = [word.lower() for word in words if word.lower() not in stop_words and word.isalpha()]
    return words_filtered

# 应用预处理
data['processed_reviews'] = data['reviews.text'].apply(preprocess_text)